In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary packages.
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
from random import sample, seed

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [4]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Rename pictures

In [ ]:
# path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/'
# comp2pakage_df = pd.read_csv(os.path.join(path, 'comp_type_to_pakage_type.csv'))
# train_df = pd.read_csv(os.path.join(path, 'training/training_set_df.csv'))
# target_comptype = comp2pakage_df[comp2pakage_df['package_type'].\
#                                  isin(['C0201','C0402','C0603','C0805','C3528','R0201','R0402','R0603'])]\
#                                  ['comp_type'].tolist()
# train_df = train_df[train_df['comp_type'].isin(target_comptype)]
# train_df['compid'] = train_df['compid'].astype('str')
# dic = {'NG':'1_', 'OK':'0_'}
# train_df['label2'] = train_df['label'].map(dic)
# train_df.insert(1, 'compid2', train_df['label2'] + train_df['compid'])

# path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/'
# for x, y in zip(train_df['compid'], train_df['compid2']):
#   old_name = path + x + '.jpg'
#   new_name = path + y + '.jpg'
#   # os.rename(old_name, new_name) # not repeat run

# Padding

In [ ]:
# Find the biggest length and width
path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/'
fname_lst = os.listdir(path)

w_lst = []
h_lst = []
for fname in tqdm(fname_lst):
  im = Image.open(os.path.join(path, fname))
  # print(im.size) # (W,H)
  w, h = im.size
  w_lst.append(w)
  h_lst.append(h)

In [ ]:
import statistics

print('width: min', min(w_lst), 'max', max(w_lst), 'median', statistics.median(w_lst), 'mean', statistics.mean(w_lst))
print('height: min', min(h_lst), 'max', max(h_lst), 'median', statistics.median(h_lst), 'mean', statistics.mean(h_lst))

width: min 60 max 1410 median 188 mean 335.84816820643516
height: min 56 max 1549 median 186 mean 321.24867792290536


In [ ]:
image = Image.open(os.path.join(path, fname_lst[200]))
image.show()

In [ ]:
# Read image sample
image = Image.open(os.path.join(path, fname_lst[200]))
w, h = image.size # (W, H)
print(image.size)
print('widht', w, 'height', h)

# Method1: independent padding to square
# padding_size = h - w
# print(padding_size)
# add_0 = abs(padding_size)//2
# if h > w:
#   pad = transforms.Pad((add_0, 0, abs(padding_size)-add_0, 0)) # left, top, right, down
# else:
#   pad = transforms.Pad((0, add_0, 0, abs(padding_size)-add_0))

# Method2: padding to the biggest length and width
w_max = 1410 
h_max = 1549
w_dif = w_max - w
h_dif = h_max - h
pad = transforms.Pad((w_dif//2, h_dif//2, w_dif-w_dif//2, h_dif-h_dif//2))

# transform sample
transform = transforms.Compose([
    pad, 
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
padded_image = transform(image)
plt.imshow(padded_image.permute(1,2,0)) # padded_image.shape # (C, H, W)

# Show OK/NG picture

In [ ]:
# demo transforms
path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/'
fname_lst = os.listdir(path)

ok_fname_lst = []
fail_fname_lst = []
for x in fname_lst:
  if '1_' in x:
    fail_fname_lst.append(x)
  if '0_' in x:
    ok_fname_lst.append(x)

In [ ]:
for fname in sample(fail_fname_lst, 10): # fail_fname_lst/ ok_fname_lst
    image = Image.open(os.path.join(path, fname))
    image.show()  

# Transforms

In [ ]:
# Transform sample
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=(89.99,90.01)),
    transforms.ColorJitter(brightness=.5,hue=0.5),
    transforms.RandomInvert(),
    transforms.ToTensor()
])
im = Image.open(os.path.join(path, ok_fname_lst[0]))
im.show()

im2 = transform(im)
plt.imshow(im2.permute(1,2,0))

In [5]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)), # resnet50, effecientNet
    transforms.ToTensor(),
])

# Training
# case1: no trans
train1_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# case2: Gaussain noise, Autocontrast and Sharpness
# train1_tfm = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.GaussianBlur(kernel_size=3),
#     transforms.ToTensor(),
# ])
# train2_tfm = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomAutocontrast(p=0.5),
#     transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
#     transforms.ToTensor(),
# ])

# Datasets

In [6]:
seed(110024519)
train_index = sample(range(12224), 10000)
valid_index = list(set(range(12224)) - set(train_index))

print(len(train_index))
print(len(valid_index))

10000
2224


In [7]:
class Task23Dataset(Dataset):

    def __init__(self, path, tfm=test_tfm, evaluation=False, train_or_valid=None): 
        super(Task23Dataset).__init__()
        self.path = path
        if evaluation == False:  # training
            self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg") and '_' in x])
        else:  # testing
            self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])

        if train_or_valid == 'train':
          self.files = list(np.array(self.files)[train_index])
        if train_or_valid == 'valid':
          self.files = list(np.array(self.files)[valid_index])
          
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
        
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        # trans = self.my_padding(im) # depend on image to padding 
        # im = trans(im) # run padding
        im = self.transform(im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im, label

    
    def my_padding(self, im):
      w, h = im.size # (W,H)
      
      # independent padding to the square of each image
      # padding_size = h - w 
      # add_0 = abs(padding_size) // 2
      # if padding_size > 0: # h > w
      #   pad = transforms.Pad((add_0, 0, abs(padding_size)-add_0, 0)) # left, top, right, down
      # else:
      #   pad = transforms.Pad((0, add_0, 0, abs(padding_size)-add_0))

      # Padding each image to the biggest length and width in all images
      w_max = 1410 
      h_max = 1549
      w_dif = w_max - w
      h_dif = h_max - h
      pad = transforms.Pad((w_dif//2, h_dif//2, w_dif-w_dif//2, h_dif-h_dif//2))

      return transforms.Compose([pad])



# Classifier model structure

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 224, 224]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 224, 224]
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2, 0),      # [64, 112, 112]
            # nn.Dropout(0.5),

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 112, 112]
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2, 0),      # [128, 56, 56]
            # nn.Dropout(0.5),

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 56, 56]
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2, 0),      # [256, 28, 28]
            # nn.Dropout(0.5),

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 28, 28]
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2, 0),       # [512, 14, 14]
            # nn.Dropout(0.5),
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 14, 14]
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2, 0),       # [512, 7, 7]
            # nn.Dropout(0.5)

        )
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(512, 2) # 2 class
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# Pre-train model
ResNet50 and EffecientNet_v2_s

In [ ]:
import torchvision.models as models

# ResNet50
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=2)

# EffecientNet
# model = models.efficientnet_v2_s(pretrained=True)
# feature_model = list(model.classifier.children())
# feature_model.pop()
# feature_model.append(nn.Linear(in_features=model.classifier[1].in_features, out_features=2))
# model.classifier = nn.Sequential(*feature_model)

#
model

# Extract features from an input image
input_image = torch.randn(1, 3, 224, 224)
features = model(input_image)
features.shape


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 133MB/s]


torch.Size([1, 2])

# Load data

In [9]:
batch_size = 64
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/'
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set1 = Task23Dataset(dataset_dir, tfm=train1_tfm, train_or_valid='train')
# train_set2 = Task23Dataset(dataset_dir, tfm=train2_tfm, train_or_valid='train')
# train_set = ConcatDataset([train_set1, train_set2])
train_loader = DataLoader(train_set1, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True) # train_set1 / train_set
valid_set = Task23Dataset(dataset_dir, tfm=test_tfm, train_or_valid='valid')
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

print(len(train_set1))
print(len(valid_set))

One /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/ sample /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/0_45301474.jpg
One /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/ sample /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/1_22833872.jpg
10000
2224


In [ ]:
im = train_set1[50][0]
print(im.shape)
plt.imshow(im.permute(1,2,0))

# Model traning

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu" 

# The number of training epochs and patience.
n_epochs = 15
patience = 300 # If no improvement in 'patience' epochs, early stop

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)

# resnet50
# pre_model = models.resnet50(pretrained=True)
# pre_model.fc = nn.Linear(in_features=pre_model.fc.in_features, out_features=2)
# model = pre_model.to(device)

# efficientnet_v2_s
# pre_model = models.efficientnet_v2_s(pretrained=True)
# feature_model = list(pre_model.classifier.children())
# feature_model.pop()
# feature_model.append(nn.Linear(in_features=pre_model.classifier[1].in_features, out_features=2))
# pre_model.classifier = nn.Sequential(*feature_model)
# model = pre_model.to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5) 
# scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1) # performance is bad

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device)) # 2x1

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()
        # scheduler.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    pred_label_lst = []
    true_label_lst = []
    missing_rate_lst = []
    benefit_lst = []

    pred_prob_lst = []


    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        
        # Compute the confusion matrix
        pred_label_lst.append(logits.argmax(dim=-1).cpu().tolist())
        true_label_lst.append(labels.tolist())
        # print(pred_label_lst)
        # print(true_label_lst)

        # softmax 
        softmax_fun = nn.Softmax(dim=1)
        output = softmax_fun(logits)
        pred_prob_lst.append(output.cpu().tolist())

        # Record the loss and accuracy.(add missing_rate and benefit.)
        valid_loss.append(loss.item())
        valid_accs.append(acc)


    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    pred_label_lst = sum(pred_label_lst, []) # un inner list
    true_label_lst = sum(true_label_lst, []) # un inner list
    TN, FP, FN, TP = confusion_matrix(true_label_lst, pred_label_lst).ravel()
    print('TN =',TN, 'FP =', FP, 'FN =', FN, 'TP =', TP) 
    missing_rate = FN / (TP+FN)
    benefit = (FN+TN) / (TP+FP+FN+TN)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] missing_rate = {missing_rate:.5f}, benefit = {benefit:.5f}")

    # update logs
    if valid_acc > best_acc:
        with open(f"./sample_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./sample_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"/content/drive/MyDrive/Colab Notebooks/task23/sample_My_CNN_best_case1.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

  0%|          | 0/157 [00:00<?, ?it/s]

[ Train | 001/015 ] loss = 0.70630, acc = 0.67655


  0%|          | 0/35 [00:00<?, ?it/s]

TN = 1244 FP = 104 FN = 369 TP = 507
[ Valid | 001/015 ] loss = 0.47974, acc = 0.78616
[ Valid | 001/015 ] missing_rate = 0.42123, benefit = 0.72527
[ Valid | 001/015 ] loss = 0.47974, acc = 0.78616 -> best
Best model found at epoch 0, saving model


  0%|          | 0/157 [00:00<?, ?it/s]

...

TN = 1298 FP = 50 FN = 115 TP = 761
[ Valid | 014/015 ] loss = 0.21453, acc = 0.92530
[ Valid | 014/015 ] missing_rate = 0.13128, benefit = 0.63534
[ Valid | 014/015 ] loss = 0.21453, acc = 0.92530


  0%|          | 0/157 [00:00<?, ?it/s]

[ Train | 015/015 ] loss = 0.13960, acc = 0.94477


  0%|          | 0/35 [00:00<?, ?it/s]

TN = 1298 FP = 50 FN = 111 TP = 765
[ Valid | 015/015 ] loss = 0.18907, acc = 0.92679
[ Valid | 015/015 ] missing_rate = 0.12671, benefit = 0.63354
[ Valid | 015/015 ] loss = 0.18907, acc = 0.92679


# Preprocess testing picture

In [10]:
path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/'
test_df = pd.read_csv(os.path.join(path, 'testing_set_df.csv'))
test_pic_fname_lst = os.listdir(os.path.join(path, 'pic'))
comp2pakage_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/comp_type_to_pakage_type.csv')
target_comptype = comp2pakage_df[comp2pakage_df['package_type'].\
                                 isin(['C0201','C0402','C0603','C0805','C3528','R0201','R0402','R0603'])]\
                                 ['comp_type'].tolist() # 2418
test_df = test_df[test_df['comp_type'].isin(target_comptype)]
test_df['compid'] = test_df['compid'].astype('str')

dic = {'NG':1, 'OK':0}
test_df['label2'] = test_df['label'].map(dic)

<ipython-input-10-63f0edfb95fa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['compid'] = test_df['compid'].astype('str')
<ipython-input-10-63f0edfb95fa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['label2'] = test_df['label'].map(dic)


In [ ]:
# # set(test_df['compid']+'.jpg').issubset(set(test_pic_fname_lst)) # True
# ## copy pic to pic_task2
# import shutil
# for x in test_df['compid']:
#   src = f'/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic/{x}.jpg'
#   dst = f'/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic_task2/{x}.jpg'
#   # shutil.copy(src, dst)

# Testing and prediction
Our model use ensemble learning with resnet50 and effecientNet

In [11]:
batch_size = 64
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic_task2/'
test_set = Task23Dataset(dataset_dir, tfm=test_tfm, evaluation=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/'
valid_set = Task23Dataset(dataset_dir, tfm=test_tfm, train_or_valid='valid')
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True) # suffle change to False

One /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic_task2/ sample /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic_task2/58672750.jpg
One /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/ sample /content/drive/MyDrive/Colab Notebooks/task23/task2_task3/training/pic/1_22833872.jpg


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
# my_cnn
model_best_mycnn = Classifier().to(device)

# resnet50
pre_model = models.resnet50(pretrained=True)
pre_model.fc = nn.Linear(in_features=pre_model.fc.in_features, out_features=2)
model = pre_model.to(device)
model_best_resnet50 = pre_model.to(device)

# efficientnet_v2_s
pre_model = models.efficientnet_v2_s(pretrained=True)
feature_model = list(pre_model.classifier.children())
feature_model.pop()
feature_model.append(nn.Linear(in_features=pre_model.classifier[1].in_features, out_features=2))
pre_model.classifier = nn.Sequential(*feature_model)
model_best_efficientnet = pre_model.to(device)

# model_best_mycnn.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/task23/sample_My_CNN_best_case1.ckpt'))
# model_best_mycnn.eval()
model_best_resnet50.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/task23/sample_resnet50_best_case1.ckpt', map_location=torch.device('cpu')))
model_best_resnet50.eval()
# model_best_efficientnet.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/task23/sample_effecientNet_best_case1.ckpt', map_location=torch.device('cpu')))
# model_best_efficientnet.eval()

prediction_prob_mycnn = []
prediction_prob_resnet50 = []
prediction_prob_efficientnet = []
softmax_fun = nn.Softmax(dim=1)
with torch.no_grad():
    for data,_ in valid_loader: # test_loader or valid_loader
        # test_pred_mycnn = model_best_mycnn(data.to(device))
        test_pred_resnet50 = model_best_resnet50(data.to(device))
        # test_pred_efficientnet = model_best_efficientnet(data.to(device))
        ## test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        ## prediction += test_label.squeeze().tolist()
        # softmax
        # output_mycnn = softmax_fun(test_pred_mycnn)
        output_resnet50 = softmax_fun(test_pred_resnet50)
        # output_effecientNet = softmax_fun(test_pred_efficientnet)
        # prediction_prob_mycnn.append(output_mycnn.cpu().tolist())
        prediction_prob_resnet50.append(output_resnet50.cpu().tolist())
        # prediction_prob_efficientnet.append(output_effecientNet.cpu().tolist())

    # prediction_prob_mycnn = sum(prediction_prob_mycnn, [])
    prediction_prob_resnet50 = sum(prediction_prob_resnet50, [])
    # prediction_prob_efficientnet = sum(prediction_prob_efficientnet, [])

Validation

In [ ]:
# Select the best threshold in validation data
def we_curve(true_label, prediction_prob, threshold=np.arange(0, 1.01, 0.001)):
  missing_rate_lst = []
  benefit_lst = []
  acc_lst = []
  for thres in threshold:
    pred_label = [1 if x[1] >= thres else 0 for x in prediction_prob] # x[1]: positive prob, If positive prob > thres, then it predict the NG(1)
    cm = confusion_matrix(true_label, pred_label) # left: true, up: pred
    TN, FP, FN, TP = cm.ravel()
    missing_rate = FN / (TP+FN)
    benefit = (FN+TN) / (TP+FP+FN+TN)
    acc = (TP+TN) / (TP+FP+FN+TN)
    missing_rate_lst.append(missing_rate)
    benefit_lst.append(benefit)
    acc_lst.append(acc)
  return missing_rate_lst, benefit_lst, acc_lst, threshold

In [ ]:
valid_label = [valid_set[i][1] for i in range(len(valid_set))] # true label
missing_rate, benefit, acc, threshold = we_curve(valid_label, prediction_prob_mycnn) # 
plt.plot(missing_rate, benefit)
plt.xlabel('missing_rate')
plt.ylabel('benefit')
plt.show()

In [ ]:
print(missing_rate[:3])
print(benefit[:3])
print(acc[:3])
print(threshold[:3])

Testing

1. Single model


In [1]:
def performance_func(true_label, pred_label):
  cm = confusion_matrix(true_label, pred_label)
  TN, FP, FN, TP = cm.ravel()
  missing_rate = FN / (TP+FN)
  benefit = (FN+TN) / (TP+FP+FN+TN)
  acc = (TP+TN) / (TP+FP+FN+TN)
  print('Confusion matrix:\n', cm)
  print(f'missing_rate: {missing_rate:.5f}, benefit: {benefit:.5f}, accuracy: {acc:.5f}')

In [ ]:
true_label = test_df['label2'].tolist() # true label
pred_label = [1 if x[1] >= 0.001 else 0 for x in prediction_prob_efficientnet] # prediction_prob_resnet50 / prediction_prob_efficientnet

performance_func(true_label, pred_label)

2. Ensemble model

In [ ]:
pred_label_resnet50 = [1 if x[1] >= 0.001 else 0 for x in prediction_prob_resnet50] # change threshold to 0.001
pred_label_efficientnet = [1 if x[1] >= 0.001 else 0 for x in prediction_prob_efficientnet]
pred_label_ensemble = []
for x, y in zip(pred_label_resnet50, pred_label_efficientnet):
  if (x==0 and y==0):
    pred_label_ensemble.append(0)
  else:
    pred_label_ensemble.append(1)

performance_func(true_label, pred_label_ensemble)

# Visualization

In [ ]:
print(len(true_label))
print(len(pred_label))

2418
2418


In [ ]:
FN_index = [x==1 and y==0 for x, y in zip(true_label, pred_label_ensemble)]
FN_fname_lst = list(test_df.iloc[FN_index]['compid']+'.jpg')

# FP_index = [x==0 and y==1 for x, y in zip(true_label, pred_label)]
# FP_fname_lst = list(test_df.iloc[FP_index]['compid']+'.jpg')

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/task23/task2_task3/testing/pic_task2/'
for fname in FN_fname_lst: # NG->ok
    print(fname)
    image = Image.open(os.path.join(path, fname))
    image.show()